# 安裝所需套件

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
# !pip install tqdm

In [ ]:
!pip install accelerate==0.2.0

# 準備模型

- 加載模型 與 tokenizer

In [27]:
import sys
sys.path.append('../FarmGPT_share/') 

In [75]:
from gpt3 import GPT3ForCausalLM, GPT3Model
from transformers import BertTokenizerFast, GenerationConfig
import torch

model = None
model = GPT3ForCausalLM.from_pretrained("HuiHuang/gpt3-damo-base-zh") # 425MB
# tokenizer = BertTokenizerFast.from_pretrained("HuiHuang/gpt3-damo-base-zh") # 預設儲存 /home/yourname/.cache/torch/transformers
tokenizer = BertTokenizerFast.from_pretrained("./LLMs/tokenizer") # 此tokenizer需要有EOS，且id為25600

print(model.num_parameters()) # 查看模型參數量
model

GPT3ForCausalLM(
  (transformer): GPT3Model(
    (word_embeddings): Embedding(25600, 768)
    (position_embeddings): Embedding(2048, 768)
    (embedding_dropout): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT3Block(
        (input_layernorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (attention): GPT3SelfAttention(
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (softmax): Softmax(dim=-1)
          (attention_dropout): Dropout(p=0.1, inplace=False)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (output_dropout): Dropout(p=0.1, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (mlp): GPT3MLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, in

In [3]:
print(torch.cuda.is_available()) 
print(torch.__version__)
print(torch.version.cuda)

True
2.0.1+cu117
11.7


In [11]:
print(tokenizer.cls_token, 'id : ', tokenizer.convert_tokens_to_ids(tokenizer.cls_token))
print(tokenizer.sep_token, 'id : ', tokenizer.convert_tokens_to_ids(tokenizer.sep_token))
print(tokenizer.eos_token, 'id : ', tokenizer.convert_tokens_to_ids(tokenizer.eos_token))
print(tokenizer.pad_token, 'id : ', tokenizer.convert_tokens_to_ids(tokenizer.pad_token))

Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


[CLS] id :  2
[SEP] id :  3
None id :  None
[PAD] id :  0


In [4]:
# 看一下效果
'''
greedy search
'''
model.eval()
input_txt = "问:今天天气如何？"
# input_txt = "问:SOC跟SIC的关联是甚麽?"
device = torch.device("cuda")
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
model = model.to(device)
# max_length數需把原問題長度也考慮進去，雖然輸出的是[CLS]問:...[SEP]答:...，但記得這個 問... 也是最後輸出的一部分
output = model.generate(input_ids, max_length=250, pad_token_id=0) #還是會出現重複字
print(tokenizer.decode(output[0]))

[CLS] 问 : 今 天 天 气 如 何 ？ [SEP] 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫 ， 我 们 家 的 猫 咪 是 一 只 小 猫 ， 我 们 家 的 猫 咪 是 一 只 大 猫


# 載入微調過的模型
- 文字生成模型在做generate()時，可透過設定參數來決定要使用greedy或beam search等策略來生成。
- 說明 : https://zhuanlan.zhihu.com/p/430961578

In [2]:
from gpt3 import GPT3ForCausalLM
from transformers import BertTokenizerFast, GenerationConfig
import torch

# 注意 : 資料夾名稱不能和模型名稱重複
model_path = "./LLMs/0827_gpt3_damo_zh"

model = None
model = GPT3ForCausalLM.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained("./LLMs/tokenizer")

model

/home/p76124388/anaconda3/envs/FarmGPT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT3ForCausalLM(
  (transformer): GPT3Model(
    (word_embeddings): Embedding(25600, 1024)
    (position_embeddings): Embedding(2048, 1024)
    (embedding_dropout): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT3Block(
        (input_layernorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (attention): GPT3SelfAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (softmax): Softmax(dim=-1)
          (attention_dropout): Dropout(p=0.1, inplace=False)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (output_dropout): Dropout(p=0.1, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (mlp): GPT3MLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(

# 創建datasets資料集物件

- 資料集處理：根據你的需求，你可能需要進一步處理資料集，例如進行分詞、編碼、切分訓練/驗證/測試集等操作。你可以使用datasets庫提供的方法進行這些操作。

- 準備資料集，先將問答資料準備成以下形式
```
data = {
    'question': ['問題1', '問題2', '問題3'],
    'answer': ['答案1', '答案2', '答案3']
}
```

In [29]:
# 先看一下資料長相
import pandas as pd
prelook_data = pd.read_csv('/home/p76124388/training_FarmGPT/dataset/ch_chatbot_arena_conversations_conbine.csv')
prelook_data.head()

,Question,Answer
0,OpenCL 和 CUDA 有什么区别？,OpenCL 和 CUDA 都是用于 GPU 上并行计算的编程语言，但它们在几个关键方面有所...
1,为什么我的父母不邀请我参加他们的婚礼？,您的父母可能出于多种原因没有邀请您参加他们的婚礼。可能是他们不想给您带来活动计划或费用的负担...
2,富士和尼康哪个更好？,这是一个主观问题，答案取决于个人喜好和需求。富士和尼康都以高品质相机而闻名，两者之间的选择最...
3,如何为聊天机器人打造竞技场？,为聊天机器人构建竞技场是鼓励用户与您的机器人交互的好方法，并为您的机器人提供展示其能力的空间...
4,今天是什么时候？,抱歉，我无法为您确定当前日期，因为我无法访问当前日期和时间。我的知识截止日期是 2021 年...


In [50]:
prelook_data['Answer'][0]

'OpenCL 和 CUDA 都是用于 GPU 上并行计算的编程语言，但它们在几个关键方面有所不同： 1. 架构：OpenCL 是通用并行计算语言，而 CUDA 是专门为 GPU 计算而设计的。 OpenCL 可以运行在各种类型的处理器上，包括 CPU、GPU 和 FPGA，而 CUDA 仅兼容 NVIDIA GPU。 2.语言：OpenCL是C++语言，而CUDA是C++语言，带有GPU计算的附加功能。 CUDA 提供了额外的库和工具来处理图形和 GPU 特有的其他任务。 3.驱动支持：OpenCL拥有更广泛的设备支持，不仅支持NVIDIA GPU，还支持AMD、Intel、ARM等其他厂商。 CUDA 仅适用于 NVIDIA GPU，并且需要在主机上安装 CUDA 工具包。 4.内存管理：OpenCL提供了统一的内存模型，所有数据都存储在全局内存中，内核可以访问其中的任何部分。另一方面，CUDA 使用分层内存架构，其中数据分为主机、设备和共享内存。 5.性能：OpenCL和CUDA在性能方面各有优缺点。 OpenCL 通常对于数值计算更快，而 CUDA 对于图形和其他需要快速内存访问的任务更快。 CUDA 还更好地支持 64 位浮点运算，这对于某些科学应用程序至关重要。综上所述，OpenCL和CUDA都是用于GPU上并行计算的编程语言，但它们在架构、语言、驱动支持、内存管理和性能方面有所不同。 OpenCL 是一种通用语言，可以在各种类型的处理器上运行，而 CUDA 特定于 NVIDIA GPU，并提供额外的库和工具来处理图形和 GPU 特有的其他任务。'

In [32]:
prelook_data[525:545]

,Question,Answer
525,碧蓝航线特典第861集“与赤城和加贺聊天！”今天的节目中，我们将与第一航母师的两艘舰赤城号和...,你好！很高兴来到这里。赤城：是的，很荣幸来到这里。加贺：我也很高兴来到这里。 Shikika...
526,碧蓝航线奖励第 238 集？ “与埃塞克斯和企业聊天！”埃塞克斯：早上好，海军上将。我是埃塞...,埃塞克斯：*傻笑*很高兴与您交谈，企业号。我必须说，我一直钦佩您的领导能力以及与团队良好合作...
527,给我讲一个关于爱丽丝和鲍勃以及素数的好故事。,曾几何时，爱丽丝和鲍勃是两个对数学充满热情的最好的朋友。他们对素数特别着迷，素数只能被它本身...
528,撰写从波多黎各到美国的独立宣言文本,我们，波多黎各人民，特此宣布脱离美利坚合众国独立。长期以来，我们一直被视为二等公民，被剥夺了...
529,如何在存储报表的数据库中快速查找包含特定项目的报表。,1. 使用报告搜索：第一步是使用报告搜索（通常位于报告界面或报告库中）来搜索报告中的特定项目...
530,担任英语教授。深入阅读一本书的第一句话：“快递员将额头压在玻璃、氩气、高强度塑料上。他看着一...,这句话的开头生动地描述了一名快递员将额头压在层层玻璃、氩气和高强度塑料上。这个描述立即为未来...
531,礼貌且专业地改述：我会再回复您有关 ABC 远程访问 XYZ 系统的问题，请参阅下面的电子邮...,亲爱的[姓名]，希望您收到此消息后一切顺利。我写信是为了询问我之前发送给您的 ABC 系统远...
532,告诉我关于法国的事,法国是西欧的一个国家。它是欧盟第三大国，仅次于德国和意大利，也是欧洲第四大国，仅次于俄罗斯、...
533,可以在Linux上使用docker设置wireguard吗,是的，可以在 Linux 上使用 Docker 设置 WireGuard。以下是执行此操作的...
534,Alex 18 岁，Steve 20 岁 Alex 20 岁时，Steve 多大？,当亚历克斯满 20 岁时，史蒂夫也满 22 岁。


# Import Packages

In [78]:
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, BertTokenizerFast, GenerationConfig
import transformers
from gpt3 import GPT3ForCausalLM, GPT3Model
import csv
from transformers import AdamW
from rouge import Rouge
import gc

""" 訓練模型 """
import nltk
from nltk.translate.meteor_score import single_meteor_score
from nltk.corpus import stopwords
import jieba
import re

from tqdm.auto import tqdm
import pickle

device = None

# 中文停用詞列表
nltk.download('stopwords') # 下載停用詞列表
nltk.download('wordnet')  # 下載同義詞資源（可選）

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
		torch.cuda.manual_seed(seed)
		torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(0)


# Change "fp16_training" to True to support automatic mixed precision training (fp16)
fp16_training = False

if fp16_training:
    # !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(mixed_precision='fp16')
    device = accelerator.device
else:
    device = "cuda"

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/p76124388/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/p76124388/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 製作dataloader

In [79]:
# 準備訓練與測試問答資料集，存於dict
train_q = []
train_a = []
# test_q = []
# test_a = []
# val_q = []
# val_a = []

# 25646

# 訓練集
with open(r'./dataset/ch_chatbot_arena_conversations_conbine.csv', 'r', encoding='utf-8', errors='ignore') as csvfile:
  reader = csv.reader(csvfile)
  next(reader) # skip title

  for i, row in enumerate(reader):
      train_q.append("问:" + row[0])
      train_a.append("答:" + row[1])
    # else:
    #   val_q.append("问:" + row[0])
    #   val_a.append("答:" + row[1])

# 驗證集 + 測試集
# with open(r'./dataset/filtered_Simplified_test_All_soilGPTdata.csv', 'r', encoding='utf-8', errors='ignore') as csvfile:
# # with open('/home/charlie200/dataset/All_mix/Simplified/filtered_Simplified_test_All_soilGPTdata.csv', 'r', encoding='utf-8', errors='ignore') as csvfile:
#   reader = csv.reader(csvfile)
#   next(reader) # 跳过第一行标题

#   # 不需要打包成像 train_data
#   for i, row in enumerate(reader):
#     if i < 500: # 限制在800筆資料
#       test_q.append("问:" + row[0])
#       test_a.append("答:" + row[1])
#     elif i < 1000:
#       val_q.append("问:" + row[0])
#       val_a.append("答:" + row[1])
#     else:
#         break
    
""" Tokenize Data """
train_q_tokenized = tokenizer(train_q, add_special_tokens=False)
# val_q_tokenized = tokenizer(val_q, add_special_tokens=False)
# test_q_tokenized = tokenizer(test_q, add_special_tokens=False)

train_a_tokenized = tokenizer(train_a, add_special_tokens=False)
# val_a_tokenized = tokenizer(val_a, add_special_tokens=False)
# test_a_tokenized = tokenizer(test_a, add_special_tokens=False)

# 列印觀察
print("train : ", len(train_q_tokenized['input_ids']))
# print("validation : ", len(val_q_tokenized['input_ids']))
# print("test : ", len(test_q_tokenized['input_ids']))

print("Q id : ", train_q_tokenized[0].ids) # 沒有自動加入[CLS]、[SEP]
print("A id : ", train_a_tokenized[0].ids)

# attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]
print("\ntrain data Q: " + str(tokenizer.decode(train_q_tokenized[0].ids, skip_special_tokens=False))) # train_q_tokenized[0].ids == train_q_tokenized['input_ids'][0]
print("train data A: " + str(tokenizer.decode(train_a_tokenized[0].ids, skip_special_tokens=False)))

train :  25646
Q id :  [18616, 40, 23363, 22445, 1878, 23845, 22464, 6611, 394, 274, 1540, 1269, 21143]
A id :  [11806, 40, 23363, 22445, 1878, 23845, 22464, 17351, 6393, 10226, 344, 24004, 212, 4416, 15126, 15979, 11873, 10574, 12768, 11477, 16055, 15562, 21124, 528, 3661, 438, 2546, 1194, 244, 1085, 18424, 6275, 18988, 6611, 5386, 215, 1750, 21138, 31, 28, 6784, 6734, 21138, 23363, 22445, 6393, 17124, 10226, 4416, 15126, 15979, 11873, 16055, 15562, 21124, 13014, 23845, 22464, 6393, 221, 18610, 260, 24004, 15979, 11873, 13014, 16008, 15979, 10574, 160, 23363, 22445, 1721, 431, 17050, 15126, 2546, 1742, 11415, 12101, 2645, 10574, 3022, 9936, 2354, 212, 21124, 1487, 5558, 22337, 159, 24004, 1878, 58, 22104, 21740, 21124, 13014, 23845, 22464, 399, 1094, 3715, 23454, 24004, 160, 32, 28, 16055, 15562, 21138, 23363, 22445, 6393, 55, 25, 25, 16055, 15562, 21124, 13014, 23845, 22464, 6393, 55, 25, 25, 16055, 15562, 21124, 4336, 6611, 24004, 15979, 11873, 10574, 18702, 1386, 1385, 13255, 160, 

In [7]:
val_q_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [5]:
# 觀察用
def get_model_predict(input_ids, model):
    model = model.to("cuda")
    print("進入get_model_predict")
    model.eval()
    with torch.no_grad():
        input_ids = [2] + input_ids
        input_ids += [3]
        input_ids = torch.tensor([input_ids]).to("cuda")
        # output = model.generate(input_ids, do_sample=True, max_length=400, top_k=35, repetition_penalty = 1.2, pad_token_id=0)
        output = model.generate(input_ids, do_sample=False, max_length=400, num_return_sequences=1, repetition_penalty = 1.4, pad_token_id=0)
        output_text = tokenizer.decode(output[0], skip_special_tokens=True) 
        output_text = output_text.partition("答 : ")[2] # 獲得輸出內容

        print('未經處理前的輸出 : ', output_text)
        return output_text
        
output = get_model_predict(train_q_tokenized[0].ids, model)

進入get_model_predict
未經處理前的輸出 :  


In [80]:
class QA_Dataset(Dataset):
    def __init__(self, split, tokenized_questions, tokenized_answers, test_answers=None, max_length=1024):
        self.split = split # "train" or "test"
        self.tokenized_questions = tokenized_questions
        self.tokenized_answers = tokenized_answers
        self.test_answers = test_answers
        self.max_length = max_length

    def __len__(self):
        return len(self.tokenized_questions['input_ids'])

    def __getitem__(self, idx):
        # return question_input_ids、answer_input_ids、attention_mask
        # 因為在模型中的 forward 會自己做 shift，所以這邊的 question_input_ids == answer_input_ids 
        tokenized_question = self.tokenized_questions[idx]
        tokenized_answer = self.tokenized_answers[idx]

        if self.split == "train":
            # add special tokens (2: CLS, 3: SEP)
            input_ids_question = [2] + tokenized_question.ids + [3]
            input_ids_answer = tokenized_answer.ids + [3]

            # Pad sequence and obtain inputs to model
            input_ids, attention_mask = self.padding(input_ids_question, input_ids_answer)

            # return (data, labels, attention mask)
            return torch.tensor(input_ids), torch.tensor(input_ids), torch.tensor(attention_mask)
        else:
            # add special tokens (2: [CLS], 3: [SEP], 11806: "答", 40: ":")
            tokenized_question = [2] + tokenized_question.ids + [3, 11806, 40]
            
            attention_mask = [1]*len(tokenized_question)

            # test_answers : 原本答案的句子。「答:计算地下水的水力梯度需要使用地下水时...」
            return torch.tensor(tokenized_question), self.test_answers[idx], torch.tensor(attention_mask)

    # 在這邊把每一筆資料長度都 padding 到 self.max_length 的長度
    def padding(self, input_ids_question, input_ids_answer):
        input_ids = input_ids_question + input_ids_answer
        attention_mask = None

        if len(input_ids) < self.max_length :
            padding_len = self.max_length - len(input_ids)
            attention_mask = [1]*len(input_ids) + [0]*padding_len
            input_ids = input_ids + [0]*padding_len
        else:
            input_ids = input_ids[:self.max_length-1] + [3]
            attention_mask = [1]*self.max_length
            
        return input_ids, attention_mask

train_set = QA_Dataset("train", train_q_tokenized, train_a_tokenized, test_answers=None)
# val_set = QA_Dataset("val", val_q_tokenized, val_a_tokenized, test_answers=val_a)
# test_set = QA_Dataset("test", test_q_tokenized, test_a_tokenized, test_answers=test_a)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True, drop_last=True)
# val_loader = DataLoader(val_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=True)
# test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=True)

print('train_loader : ', len(train_loader))
# print('val_loader : ', len(val_loader))

import gc

# del train_q_tokenized, train_a_tokenized, val_q_tokenized, val_a_tokenized, train_q, train_a
del train_q_tokenized, train_a_tokenized, train_q, train_a
gc.collect()

train_loader :  3205


50

# 觀察 : DataCollatorWithPadding 可做自動`<PAD>`填充

- 會檢查這些樣本中具有最大長度的特徵，並將其他樣本的相應特徵進行填充，使得整個批次中的所有樣本具有相同長度的特徵。

- 並且使用特殊的填充標記（如 `<PAD>`）對序列進行填充，並生成一個具有相同長度的批次。這樣，就可以將這個批次作為模型的輸入，進行批次並行計算和訓練。

In [ ]:
# huggingface中的data_loader : DataCollatorWithPadding (讓資料可以一次加載一個batch)
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 每一個batch中，每筆資料拓展到跟當中一樣長度

In [ ]:
""" 經過DataCollatorWithPadding之前 """
samples = tokenized_datasets['train'][:5] # 取出前5筆資資料
samples = {k: v for k, v in samples.items() if k not in ['question', 'answer']} # 不需要處理前的資料了
[len(x) for x in samples["input_ids"]] # 每一筆sample的長度

In [ ]:
""" 經過DataCollatorWithPadding之後，所有樣本長度都是固定的 """
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()} # k:樣本數、v:一筆樣本padding之後的長度

In [ ]:
tokenized_datasets['train']['input_ids'][0]

In [ ]:
tokenizer.vocab_size

# (opt)定義下游任務模型
- lm_head : 用來執行"生成文字"的工作
- huggingface上面的文字生成序列模型主要都是做自回歸，所以在他們設定的loss那邊已經有做位移一位的處理。若要將此gpt用於其他任務，需要重新定義loss等。

In [ ]:
from gpt3 import GPT3ForCausalLM, GPT3Model

class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.pretrained = AutoModel.from_pretrained("模型名")
    self.fc = torch.nn.Linear(768, tokenizer.vocab_size, bias=False)
    self.criterion = torch.nn.CrossEntropyLoss()
    ...

  # 根據下游任務自訂
  def forward(self, input_ids, attention_mask, labels):
    lm_logits = self.pretrained(input_ids=input_ids, attention_mask=attention_mask)
    ...
    
  # 根據下游任務自訂
  def generate(self, text):
    ...

# 訓練模型
[參考影片(08:14左右)](https://www.bilibili.com/video/BV1zt4y1u7Vy?p=3&vd_source=9661bf9273e6be67d0006b48ca21db55)

[參考影片2](https://www.bilibili.com/video/BV1Kv4y157Su?p=5&vd_source=9661bf9273e6be67d0006b48ca21db55)

In [81]:
def get_model_output(model, val_loader, needSpace=False):
  global device
  with torch.no_grad():
    model.eval()
    model = model.to(device)
    output_texts = []
    
    for batch_idx, data in enumerate(tqdm(val_loader)):
      # data[0] : tokenized_Q ; data[1] : ground truth("答:土壤...") ; data[2] : attention mask(全1的list)
      data[0] = data[0].to(device)
      data[2] = data[0].to(device)

#       output = model.generate(data[0], do_sample=False, max_length=150, attention_mask=data[2], pad_token_id=0)
#       output = model.generate(data[0], do_sample=True, max_length=150, top_k=50)
      # greedy
      output = model.generate(data[0], do_sample=False, max_length=150, num_return_sequences=1, repetition_penalty = 1.2, pad_token_id=0)
      output_text = tokenizer.decode(output[0], skip_special_tokens=False) # tokenizer.decode() 的輸出會在每個字中間自動加入空白格

      # 使用 split() 方法將文本按空格分割成單詞列表
      word_list = output_text.split()
      sep_index = word_list.index("[SEP]")

      # 從 [SEP] 的下一個詞開始提取文字(取「答:」部分)，並加入空白
      if (needSpace):
        output_text = ' '.join(word_list[sep_index + 1:])
      else:
        output_text = ''.join(word_list[sep_index + 1:])

      # 為了計算評估指標，如ROUGE等，所以把最後的特殊token([SEP])拔掉
      output_text = re.sub(r"\[SEP\]$", "", output_text)
      output_texts.append(output_text)

    print("預覽 : ", output_texts[0])
    
    del output_text, word_list
    gc.collect()
    return output_texts

# 回傳([單字list1, 單字list2, ...], [加入空白格的句子1, 加入空白格的句子2, ...]) 
def tokenizeChineseText(sentences):
  arr_word_list = []
  arr_sentence_list = []
  chinese_stopwords = stopwords.words('chinese')
  for i in range(len(sentences)):
    word_list = list(jieba.cut(sentences[i]))  # 使用結巴分詞，斷成如 ['答', ':', '生物', '的', '生存'] 樣式

    # 過濾停用詞
    word_list = [word for word in word_list if word not in chinese_stopwords]
    arr_word_list.append(word_list)
    arr_sentence_list.append(' '.join(word_list)) # 變成每個"單字"之間有包含空格

  return arr_word_list, arr_sentence_list


# 計算 ROUGE、METEOR 分數
def count_criterion(output_texts, val_loader):
  rouge = Rouge()
  rouge_avg = 0.0
  meteor_avg = 0.0
  ground_truth = [data[1][0] for data in val_loader]
        
  tokenized_output_words, tokenized_output_sentences = tokenizeChineseText(output_texts)
  tokenized_ground_truth_words, tokenized_ground_truth_sentences = tokenizeChineseText(ground_truth)

  """ ROUGE """
  # 需要使用 【'我 今 天'】 這種格式
  rouge_avg = rouge.get_scores(tokenized_output_sentences, tokenized_ground_truth_sentences, avg=True)['rouge-l']['f']

  """ METEOR """
  # 需要使用 【 ['我', '今天'] 】 這種格式
  for i in range(len(output_texts)):
    meteor_avg += single_meteor_score(tokenized_ground_truth_words[i], tokenized_output_words[i])
  meteor_avg /= len(output_texts)

  return rouge_avg, meteor_avg

# 微調模型
def train(model, num_epochs, optimizer, scheduler, train_loader, val_loader, accum_iter, early_stop_epoch, save_path):
  global device
  model.to(device)
  max_rouge_avg = 0.0
  max_meteor_avg = 0.0
  # early_stop_counter = 0
  
  for epoch in range(num_epochs):
    total_loss = 0.0
    total_acc = 0.0

    for batch_idx, data in enumerate(tqdm(train_loader)):
      # Load all data into GPU
      data = [i.to(device) for i in data]  # data : (question_input_ids、answer_input_ids、attention_mask)
      model.train()
      output = model(data[0], labels=data[1], attention_mask=data[2]) # 此處是做自回歸，所以label相同 # 從code中看起來有自動將label移位，所以不須自己將label移動一位啦
      accum_loss = output.loss / accum_iter
      total_loss += output.loss.item() # 為了下面輸出用的~

      if fp16_training:
        accelerator.backward(accum_loss)
      else:
        accum_loss.backward()
                
      # weights update
      if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
        optimizer.step()
        scheduler.step() 
        optimizer.zero_grad()
        
      labels = data[0][..., 1:].to("cpu")  # 此處是做自回歸，所以label相同 # 從code中看起來有自動將label移位，所以不須自己將label移動一位啦
      out = output.logits.argmax(dim=2)[..., :-1].to("cpu")
      total_acc += (labels == out).sum().item() / labels.numel() # numel : 獲取tensor中一共包含多少個元素
      
    # 計算預測下個字的準確度
    average_loss = total_loss / len(train_loader)
    average_acc = total_acc / len(train_loader)
    
    info_out = f"Epoch {epoch+1}: Average Loss = {average_loss:.4f} Accuracy = {average_acc:.4f}"
    print(info_out)
    # file_record.write(str(info_out) + str('\n'))

    # 使用測試資料，計算 ROUGE
    if (epoch+1) % 2 == 0:
    #   print('模型驗證中...')
    #   output_texts = get_model_output(model, val_loader, needSpace=False)
    #   rouge_avg, meteor_avg = count_criterion(output_texts, val_loader)
    #   print(f"rouge_avg = {rouge_avg:.4f} meteor_avg = {meteor_avg:.4f}")

    #   if rouge_avg > max_rouge_avg and meteor_avg > max_meteor_avg:
    #     max_rouge_avg = rouge_avg
    #     max_meteor_avg = meteor_avg
    #     model.save_pretrained(save_path)
    #     early_stop_counter = 0

    #     # 保存 optimizer 的狀態
    #     with open(save_path + '/optimizer_state.pkl', 'wb') as f:
    #         pickle.dump(optimizer.state_dict(), f)

    #     # file_record.write("\n儲存模型!  rouge : " + str(rouge_avg) + ", meteor : " + str(meteor_avg) + '\n')
    #     print("儲存模型!  rouge : " + str(rouge_avg) + ", meteor : " + str(meteor_avg))
    #   else:
    #     early_stop_counter += 1

      # 直接儲存模型
      print("儲存模型")
      model.save_pretrained(save_path)
      # 保存 optimizer 的狀態
      with open(save_path + '/optimizer_state.pkl', 'wb') as f:
          pickle.dump(optimizer.state_dict(), f)
    
    gc.collect()
    torch.cuda.empty_cache()

    # if early_stop_counter == early_stop_epoch:
    #   print("early stop!")
    #   # file_record.write("early stop!" + str('\n'))
    #   break

In [88]:
# 設定訓練參數
learning_rate = 6e-4 # base : 6.0e-4, large: 3e-4
num_epochs = 7 # 14 -> 8
accum_iter = 16 # 累積4次才做參數更新
early_stop_epoch = 3    # 若經過 early_stop_epoch*2 個 epochs 沒有儲存模型，就停止訓練  # 3 -> 2
continue_training = True

num_warmup_steps = 100
def noam_lambda(current_step: int):
    current_step += 1
    return min(current_step ** (-0.5), current_step * num_warmup_steps ** (-1.5))
    
# 定義優化器 and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01) # weight_decay使用L2正則化防止過擬合

# 重新加載優化器資料，優化器在儲存時會保存先前訓練到哪個 step，所以就算有用 scheduler 也沒關係，會接下去訓練
if continue_training:
  import pickle
  with open('./LLMs/0913_chat_gpt3_base_zh/optimizer_state.pkl', 'rb') as f:
    optimizer_state = pickle.load(f)
    optimizer.load_state_dict(optimizer_state)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9) # 原本是 0.9
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = noam_lambda)

if fp16_training:
    model, optimizer, train_loader, scheduler = accelerator.prepare(model, optimizer, train_loader, scheduler)

# 開始訓練
print("開始訓練~")
save_path = "./LLMs/0913_chat_gpt3_base_zh"
val_loader = None
train(model, num_epochs, optimizer, scheduler, train_loader, val_loader, accum_iter, early_stop_epoch, save_path)

# 保存 optimizer 的狀態
with open(save_path + '/optimizer_state.pkl', 'wb') as f:
    pickle.dump(optimizer.state_dict(), f)
model.save_pretrained(save_path + "/final")

# file_record.close()

/home/p76124388/anaconda3/envs/FarmGPT/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


開始訓練~


100%|██████████| 3205/3205 [30:12<00:00,  1.77it/s]


Epoch 1: Average Loss = 0.7116 Accuracy = 0.8584


100%|██████████| 3205/3205 [30:14<00:00,  1.77it/s]


...

Epoch 5: Average Loss = 0.6818 Accuracy = 0.8625


100%|██████████| 3205/3205 [30:15<00:00,  1.77it/s]


Epoch 6: Average Loss = 0.6777 Accuracy = 0.8632
儲存模型


100%|██████████| 3205/3205 [30:15<00:00,  1.76it/s]


Epoch 7: Average Loss = 0.6740 Accuracy = 0.8638
